In [61]:
#Importação das bibliotecas utilizadas
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL da página
url = 'https://steamdb.info/sales/'

# Cabeçalhos da requisição, caso não encontre, o site irá banir seu IP
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'Cache-Control': 'max-age=0',
    'Cookie': 'cf_clearance=sIvqTUf6BFe5V_xONZu8SIPi2ZTukyyDbPY1HMAro5k-1688344842-0-160; __cf_bm=G7nU3oNoZfnaDe1L8N3J.f5fvREJdzm0x9_DuE0wznU-1688513997-0-AaxhlyiULh+y3TO90Y5TMNrzv6s69Mhe0Vkeh2nP/zEm/kS38+l7RnTZ1a5/STWkRlGz6EjzCP3dsygcEJnlsbbT2HcSI5w+XMBwaX4xJQf8',
    'Sec-Ch-Ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
    'Sec-Ch-Ua-Mobile': '?0',
    'Sec-Ch-Ua-Platform': '"Windows"',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'cross-site',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

# Faz a requisição para a página com os cabeçalhos
response = requests.get(url, headers=headers)

# Obtém o conteúdo HTML da resposta
html = response.content

# Analisa o HTML usando o BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')


game_items = soup.find_all('tr', class_='app')
    
#game_items - Apenas para teste

titulos = []
precos = []
#Extraindo o preço e o titulo dos games da pagina Sales
elementos = soup.find_all('td', class_='applogo')
for elemento in elementos:
    titulo_elemento = elemento.find_next('td').find('a', class_='b')
    if titulo_elemento:
        titulo = titulo_elemento.text.strip()
        titulos.append(titulo)

    preco_elemento = elemento.find_next('td').find_next('td').find_next('td')
    if preco_elemento:
        preco = preco_elemento.text.strip()
        precos.append(preco)

#Preparando os dados para criar um DataFrame Pandas
Dados = {'Título' : titulos,'Preço' : precos}

#Criando o DataFrame
df = pd.DataFrame(Dados)

#df - Apenas a titulo de visualização



,Título,Preço
0,Counter-Strike,"R$ 2,06"
1,Team Fortress Classic,"R$ 1,08"
2,Day of Defeat,"R$ 1,08"
3,Deathmatch Classic,"R$ 1,08"
4,Half-Life: Opposing Force,"R$ 1,08"
...,...,...
4327,Tom Clancy's Ghost Recon® Breakpoint,"R$ 35,80"
4328,September 7th,"R$ 4,92"
4329,Riders Republic,"R$ 62,47"
4330,Far Cry® 6,"R$ 62,49"


In [87]:
df.to_csv('DataFrameSteamDB',index=False) # Criando um documento para caso a importação via código para o GBQ dê errado

In [66]:
#Instalando bibliotecas necessárias para a integração com o GBQ
!pip install google-cloud-bigquery
!pip install pandas-gbq



     -------------------------------------- 21.5/21.5 MB 720.4 kB/s eta 0:00:00
     -------------------------------------- 190.7/190.7 kB 2.3 MB/s eta 0:00:00
     -------------------------------------- 151.7/151.7 kB 9.4 MB/s eta 0:00:00


In [80]:
#Chamando as Bibliotecas
from google.cloud import bigquery
from google.oauth2 import service_account

In [84]:
#Criando Credenciais
key_path= "teste-beanalytics-1a5eb83c42cb.json"
credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform"])

In [85]:
#Criando a tabela no GBQ
df.to_gbq(credentials=credentials,destination_table='deployPython.DataFrameSteamDB',if_exists='replace')

Forbidden: 403 GET https://bigquery.googleapis.com/bigquery/v2/projects/teste-beanalytics/datasets/deployPython/tables/DataFrame?prettyPrint=false: Access Denied: Table teste-beanalytics:deployPython.DataFrame: Permission bigquery.tables.get denied on table teste-beanalytics:deployPython.DataFrame (or it may not exist).